In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split

## 模型训练

In [7]:
def lgb_pred():
    # 读取训练集和测试集
    train_df=pd.read_csv('data/train.csv')
    test_df=pd.read_csv('data/test.csv')
    
    # 异常值清洗,从1.94576上升到1.90965
    train_df = train_df.drop(train_df[(train_df['房屋面积']>0.1)].index)
    
    drop_cols=['小区房屋出租数量','时间','地铁线路'] # 去掉的特征列'SubwayLine','RentRoom','Time'
    # 对房屋方向进行labelEncoder
    try:
        from sklearn.preprocessing import LabelEncoder
        lb_encoder=LabelEncoder()
        lb_encoder.fit(train_df.loc[:,'房屋朝向'].append(test_df.loc[:,'房屋朝向'])) 
        train_df.loc[:,'房屋朝向']=lb_encoder.transform(train_df.loc[:,'房屋朝向'])
        test_df.loc[:,'房屋朝向']=lb_encoder.transform(test_df.loc[:,'房屋朝向'])
    except Exception as e:
        print(e)
    
    #构造相对高度、房屋总数量、和房屋平均面积
#     train_df.loc[:,'相对高度']=train_df.loc[:,'楼层']/(train_df.loc[:,'总楼层']+1)
#     test_df.loc[:,'相对高度']=test_df.loc[:,'楼层']/(test_df.loc[:,'总楼层']+1)
#     train_df.loc[:,'总房间数']=train_df.loc[:,'厅的数量']+train_df.loc[:,'卧室数量']+train_df.loc[:,'卫的数量']
#     test_df.loc[:,'总房间数']=test_df.loc[:,'厅的数量']+test_df.loc[:,'卧室数量']+test_df.loc[:,'卫的数量']
#     train_df.loc[:,'Area/Room']=train_df.loc[:,'房屋面积']/(train_df.loc[:,'总房间数']+1)
#     test_df.loc[:,'Area/Room']=test_df.loc[:,'房屋面积']/(test_df.loc[:,'总房间数']+1)
    train_df['相对高度']=train_df['楼层'] / (train_df['总楼层'] + 1)
    test_df['相对高度']=test_df['楼层'] / (test_df['总楼层'] + 1)
    train_df['总房间数']=train_df['卧室数量'] + train_df['厅的数量'] + train_df['卫的数量']
    test_df['总房间数']=test_df['卧室数量'] + test_df['厅的数量'] + test_df['卫的数量']
    train_df['Area/Room']=train_df['房屋面积'] / (train_df['总房间数']+1)
    test_df['Area/Room']=test_df['房屋面积'] / (test_df['总房间数']+1)

    
    # 对商圈位置、地铁线路按照租金均值进行rank并作为其label
    rank_cols=['位置','地铁线路']
    for col in rank_cols:
        # 对df按照当前的col特征列进行分组，求每个组的平均值，并按照'Rental'从小到大的顺序进行排序，
        rank_df=train_df.loc[:,[col,'月租金']].groupby(col,as_index=False).mean().sort_values(by='月租金').reset_index(drop=True)
        rank_df.loc[:,col+'_rank']=rank_df.index+1        # +1，为缺失值预留一个0值的rank
        rank_fe_df=rank_df.drop(['月租金'],axis=1)
        # 将rank_fe_df按照col列合并入train_df&test_df
        train_df=train_df.merge(rank_fe_df,how='left',on=col)
        test_df=test_df.merge(rank_fe_df,how='left',on=col)
        # 删除当前的col列
        try:
            train_df.drop([col],axis=1,inplace=True)
            test_df.drop([col],axis=1,inplace=True)
        except Exception as e:
            print(e)
            
    # 删除drop_cols中的特征列
    for drop_col in drop_cols:
            try:
                train_df.drop(drop_col,axis=1,inplace=True)
                test_df.drop(drop_col,axis=1,inplace=True)
            except Exception as e:
                pass
            
    
    ######################################################
    ## 1.904/0.754408后的特征测试
    ######################################################
    #     # 卧室厅、卧室卫、厅卫
    #     train_df['BedLivingroom']=train_df['厅的数量']+train_df['卧室数量']
    #     test_df['BedLivingroom']=test_df['厅的数量']+test_df['卧室数量']
    #     train_df['DedBathroom']=train_df['卫的数量']+train_df['卧室数量']
    #     test_df['DedBathroom']=test_df['卫的数量']+test_df['卧室数量']
    #     train_df['LivBathroom']=train_df['卫的数量']+train_df['厅的数量']
    #     test_df['LivBathroom']=test_df['卫的数量']+test_df['厅的数量']
    # 卧室占比、厅占比、卫占比
    #     train_df['BedroomRitio']=train_df['卧室数量']/train_df['总房间数']
    #     test_df['BedroomRitio']=test_df['卧室数量']/test_df['总房间数']
    #     train_df['LivingRitio']=train_df['厅的数量']/train_df['总房间数']
    #     test_df['LivingRitio']=test_df['厅的数量']/test_df['总房间数']   
    #     train_df['BathRitio']=train_df['卫的数量']/train_df['总房间数']
    #     test_df['BathRitio']=test_df['卫的数量']/test_df['总房间数']    
    #卧室面积、厅的面积、卫的面积 
    train_df['卧室面积']=train_df['房屋面积']*(train_df['卧室数量']/train_df['总房间数'])
    test_df['卧室面积']=test_df['房屋面积']*(test_df['卧室数量']/test_df['总房间数'])  
    train_df['厅的面积']=train_df['房屋面积']*(train_df['厅的数量']/train_df['总房间数'])
    test_df['厅的面积']=test_df['房屋面积']*(test_df['厅的数量']/test_df['总房间数'])   
    train_df['卫的面积']=train_df['房屋面积']*(train_df['卫的数量']/train_df['总房间数'])
    test_df['卫的面积']=test_df['房屋面积']*(test_df['卫的数量']/test_df['总房间数'])
    #     train_df['else']=train_df['房屋面积'] - train_df['卧室数量']
    #     test_df['else']=test_df['房屋面积'] - test_df['卧室数量']
    ######################################################
    ## 1.906/0.738673后的特征测试
    ######################################################
    #     train_df['距离'] = train_df['距离'].fillna(1.)
    #     test_df['距离'] = test_df['距离'].fillna(1.)
    #     def fill_na(df):
    #         a=df['地铁线路_rank'].dropna().mode()
    #         df['地铁线路_rank'] = df['地铁线路_rank'].fillna(a)
    #         a=df['地铁站点'].dropna().mode()
    #         df['地铁站点'] = df['地铁站点'].fillna(a)
    #         return df
    #     train_df = train_df.groupby('小区名').apply(lambda x:fill_na(x))
    #     test_df = test_df.groupby('小区名').apply(lambda x:fill_na(x))
    #     # 对小区、区、商圈的角度来进行构建特征
    #     items = ['房屋面积','卧室数量','厅的数量','卫的数量','总楼层']
    #     for item in items:
    #         temp_train = train_df.groupby('小区名', as_index=False)[item].agg({item+'sum1':'sum'})
    #         temp_test = test_df.groupby('小区名', as_index=False)[item].agg({item+'sum1':'sum'})
    #     train_df.merge(temp_train, on='小区名', how='left')
    #     test_df.merge(temp_test, on='小区名', how='left')

    #     items = ['房屋面积','卧室数量','厅的数量','卫的数量','总楼层']
    #     for item in items:
    #         temp_train = train_df.groupby('区', as_index=False)[item].agg({item+'sum2':'sum'})
    #         temp_test = test_df.groupby('区', as_index=False)[item].agg({item+'sum2':'sum'})
    #     train_df.merge(temp_train, on='区', how='left')
    #     test_df.merge(temp_test, on='区', how='left')

    # 统计小区附近的地铁站点（效果明显）
    temp = train_df.groupby('小区名')['地铁站点'].count().reset_index()
    temp.columns = ['小区名','地铁站点数量']
    train_df = train_df.merge(temp, how = 'left',on = '小区名')
    test_df = test_df.merge(temp, how = 'left',on = '小区名')

    temp1 = train_df.groupby('小区名')['地铁线路_rank'].count().reset_index()
    temp1.columns = ['小区名','地铁线路数量']
    train_df = train_df.merge(temp1, how = 'left',on = '小区名') 
    test_df = test_df.merge(temp1, how = 'left',on = '小区名')
    ######################################################
    ## 1.87246/0.72992后的特征测试
    ######################################################
    train_df['PerFloorBedroomArea'] = train_df['相对高度'] * train_df['卧室面积']
    test_df['PerFloorBedroomArea'] = test_df['相对高度'] * test_df['卧室面积']
    #     train_df['PerFloorArea'] = train_df['相对高度'] * train_df['卧室面积']
    #     test_df['PerFloorArea'] = test_df['相对高度'] * test_df['卧室面积']
    
    # 构造训练、测试集
    X_train=train_df.drop(['月租金'],axis=1)
    Y_train=train_df.loc[:,'月租金'].values
    test_id=test_df.loc[:,'id']
    X_test=test_df.drop(['id'],axis=1)
    model_lgb = lgb.LGBMRegressor(objective='regression',  # 目标函数：回归
                                  num_leaves=900, # 叶子节点个数
                                  learning_rate=0.1, # 学习率
                                  n_estimators=3141, # 最佳迭代轮数
                                  bagging_fraction=0.7, # 建树的样本采样比例
                                  feature_fraction=0.6, # 建树的特征选择比例
                                  reg_alpha=0.3, # L1正则化
                                  reg_lambda=0.3, # L2正则化
                                  min_data_in_leaf=18, 
                                  min_sum_hessian_in_leaf=0.001)

    model_lgb.fit(X_train, Y_train,
                  eval_set=[(X_train, Y_train)])
    
    y_pred = model_lgb.predict(X_test)

    sub_df=pd.DataFrame({
        'id':test_id,
        'price':y_pred
    })
    sub_df.to_csv('./result/lgb.csv',index=False)
    print('Done!')

    return None

lgb_pred()

[1]	training's l2: 33.5719
[2]	training's l2: 28.4093
[3]	training's l2: 24.2995
[4]	training's l2: 20.7106
[5]	training's l2: 17.7008
[6]	training's l2: 15.4245
[7]	training's l2: 13.3941
[8]	training's l2: 11.6876
[9]	training's l2: 10.3796
[10]	training's l2: 9.28257
[11]	training's l2: 8.40402
[12]	training's l2: 7.51639
[13]	training's l2: 6.84773
[14]	training's l2: 6.25379
[15]	training's l2: 5.73237
[16]	training's l2: 5.29805
[17]	training's l2: 4.89423
[18]	training's l2: 4.55953
[19]	training's l2: 4.27521
[20]	training's l2: 4.00829
[21]	training's l2: 3.7809
[22]	training's l2: 3.60462
[23]	training's l2: 3.43521
[24]	training's l2: 3.28256
[25]	training's l2: 3.15025
[26]	training's l2: 3.02905
[27]	training's l2: 2.90609
[28]	training's l2: 2.80145
[29]	training's l2: 2.71063
[30]	training's l2: 2.62128
[31]	training's l2: 2.54721
[32]	training's l2: 2.48265
[33]	training's l2: 2.41846
[34]	training's l2: 2.35372
[35]	training's l2: 2.29904
[36]	training's l2: 2.24666
[3

[283]	training's l2: 0.519436
[284]	training's l2: 0.51856
[285]	training's l2: 0.517424
[286]	training's l2: 0.516279
[287]	training's l2: 0.514914
[288]	training's l2: 0.513536
[289]	training's l2: 0.512264
[290]	training's l2: 0.511496
[291]	training's l2: 0.510153
[292]	training's l2: 0.509426
[293]	training's l2: 0.508235
[294]	training's l2: 0.507406
[295]	training's l2: 0.506594
[296]	training's l2: 0.505101
[297]	training's l2: 0.504163
[298]	training's l2: 0.503185
[299]	training's l2: 0.502222
[300]	training's l2: 0.501359
[301]	training's l2: 0.499892
[302]	training's l2: 0.498996
[303]	training's l2: 0.497336
[304]	training's l2: 0.496622
[305]	training's l2: 0.49582
[306]	training's l2: 0.494235
[307]	training's l2: 0.493077
[308]	training's l2: 0.492081
[309]	training's l2: 0.4909
[310]	training's l2: 0.489954
[311]	training's l2: 0.489099
[312]	training's l2: 0.488312
[313]	training's l2: 0.487384
[314]	training's l2: 0.486537
[315]	training's l2: 0.486022
[316]	training

[559]	training's l2: 0.369469
[560]	training's l2: 0.369286
[561]	training's l2: 0.369036
[562]	training's l2: 0.368839
[563]	training's l2: 0.368648
[564]	training's l2: 0.368412
[565]	training's l2: 0.368205
[566]	training's l2: 0.368017
[567]	training's l2: 0.367801
[568]	training's l2: 0.367694
[569]	training's l2: 0.367523
[570]	training's l2: 0.367097
[571]	training's l2: 0.366851
[572]	training's l2: 0.366688
[573]	training's l2: 0.366467
[574]	training's l2: 0.366108
[575]	training's l2: 0.365866
[576]	training's l2: 0.365725
[577]	training's l2: 0.365441
[578]	training's l2: 0.365126
[579]	training's l2: 0.364971
[580]	training's l2: 0.364796
[581]	training's l2: 0.364509
[582]	training's l2: 0.364257
[583]	training's l2: 0.36405
[584]	training's l2: 0.363773
[585]	training's l2: 0.363628
[586]	training's l2: 0.363335
[587]	training's l2: 0.363136
[588]	training's l2: 0.362989
[589]	training's l2: 0.362742
[590]	training's l2: 0.362569
[591]	training's l2: 0.36238
[592]	traini

[835]	training's l2: 0.329339
[836]	training's l2: 0.329283
[837]	training's l2: 0.329203
[838]	training's l2: 0.329139
[839]	training's l2: 0.32902
[840]	training's l2: 0.328909
[841]	training's l2: 0.328822
[842]	training's l2: 0.328771
[843]	training's l2: 0.328721
[844]	training's l2: 0.328626
[845]	training's l2: 0.328507
[846]	training's l2: 0.328454
[847]	training's l2: 0.328364
[848]	training's l2: 0.328268
[849]	training's l2: 0.328102
[850]	training's l2: 0.327995
[851]	training's l2: 0.327924
[852]	training's l2: 0.327879
[853]	training's l2: 0.327805
[854]	training's l2: 0.32774
[855]	training's l2: 0.32766
[856]	training's l2: 0.327605
[857]	training's l2: 0.327546
[858]	training's l2: 0.327503
[859]	training's l2: 0.327446
[860]	training's l2: 0.32732
[861]	training's l2: 0.327267
[862]	training's l2: 0.32719
[863]	training's l2: 0.327139
[864]	training's l2: 0.327047
[865]	training's l2: 0.32697
[866]	training's l2: 0.326883
[867]	training's l2: 0.326815
[868]	training's

[1107]	training's l2: 0.314234
[1108]	training's l2: 0.314206
[1109]	training's l2: 0.314177
[1110]	training's l2: 0.314114
[1111]	training's l2: 0.314092
[1112]	training's l2: 0.314051
[1113]	training's l2: 0.314021
[1114]	training's l2: 0.313988
[1115]	training's l2: 0.313964
[1116]	training's l2: 0.313927
[1117]	training's l2: 0.313901
[1118]	training's l2: 0.313867
[1119]	training's l2: 0.313842
[1120]	training's l2: 0.313819
[1121]	training's l2: 0.313783
[1122]	training's l2: 0.31376
[1123]	training's l2: 0.3137
[1124]	training's l2: 0.313673
[1125]	training's l2: 0.31365
[1126]	training's l2: 0.31362
[1127]	training's l2: 0.313581
[1128]	training's l2: 0.313546
[1129]	training's l2: 0.313511
[1130]	training's l2: 0.313461
[1131]	training's l2: 0.313421
[1132]	training's l2: 0.313363
[1133]	training's l2: 0.313332
[1134]	training's l2: 0.313309
[1135]	training's l2: 0.313279
[1136]	training's l2: 0.313236
[1137]	training's l2: 0.313192
[1138]	training's l2: 0.313154
[1139]	traini

...

[1907]	training's l2: 0.301436
[1908]	training's l2: 0.301432
[1909]	training's l2: 0.301421
[1910]	training's l2: 0.301414
[1911]	training's l2: 0.30141
[1912]	training's l2: 0.301406
[1913]	training's l2: 0.301402
[1914]	training's l2: 0.301395
[1915]	training's l2: 0.30139
[1916]	training's l2: 0.301383
[1917]	training's l2: 0.30138
[1918]	training's l2: 0.301374
[1919]	training's l2: 0.301364
[1920]	training's l2: 0.301362
[1921]	training's l2: 0.301355
[1922]	training's l2: 0.30135
[1923]	training's l2: 0.301347
[1924]	training's l2: 0.301343
[1925]	training's l2: 0.30134
[1926]	training's l2: 0.301335
[1927]	training's l2: 0.301332
[1928]	training's l2: 0.301323
[1929]	training's l2: 0.301313
[1930]	training's l2: 0.301306
[1931]	training's l2: 0.301303
[1932]	training's l2: 0.301296
[1933]	training's l2: 0.30129
[1934]	training's l2: 0.301283
[1935]	training's l2: 0.30128
[1936]	training's l2: 0.301277
[1937]	training's l2: 0.301272
[1938]	training's l2: 0.301268
[1939]	training

[2173]	training's l2: 0.300268
[2174]	training's l2: 0.300265
[2175]	training's l2: 0.300263
[2176]	training's l2: 0.300261
[2177]	training's l2: 0.300256
[2178]	training's l2: 0.300254
[2179]	training's l2: 0.300249
[2180]	training's l2: 0.300247
[2181]	training's l2: 0.300243
[2182]	training's l2: 0.30024
[2183]	training's l2: 0.300236
[2184]	training's l2: 0.300233
[2185]	training's l2: 0.300229
[2186]	training's l2: 0.300227
[2187]	training's l2: 0.300224
[2188]	training's l2: 0.300222
[2189]	training's l2: 0.300219
[2190]	training's l2: 0.300217
[2191]	training's l2: 0.300213
[2192]	training's l2: 0.300211
[2193]	training's l2: 0.300208
[2194]	training's l2: 0.300206
[2195]	training's l2: 0.300204
[2196]	training's l2: 0.300199
[2197]	training's l2: 0.300197
[2198]	training's l2: 0.300194
[2199]	training's l2: 0.300192
[2200]	training's l2: 0.300189
[2201]	training's l2: 0.300187
[2202]	training's l2: 0.300183
[2203]	training's l2: 0.300181
[2204]	training's l2: 0.30018
[2205]	tra

[2440]	training's l2: 0.299566
[2441]	training's l2: 0.299562
[2442]	training's l2: 0.29956
[2443]	training's l2: 0.299557
[2444]	training's l2: 0.299553
[2445]	training's l2: 0.299551
[2446]	training's l2: 0.299547
[2447]	training's l2: 0.299545
[2448]	training's l2: 0.299543
[2449]	training's l2: 0.299542
[2450]	training's l2: 0.299541
[2451]	training's l2: 0.299539
[2452]	training's l2: 0.299537
[2453]	training's l2: 0.299534
[2454]	training's l2: 0.299531
[2455]	training's l2: 0.29953
[2456]	training's l2: 0.299528
[2457]	training's l2: 0.299526
[2458]	training's l2: 0.299524
[2459]	training's l2: 0.299521
[2460]	training's l2: 0.299519
[2461]	training's l2: 0.299517
[2462]	training's l2: 0.299515
[2463]	training's l2: 0.299514
[2464]	training's l2: 0.299512
[2465]	training's l2: 0.299511
[2466]	training's l2: 0.299508
[2467]	training's l2: 0.299507
[2468]	training's l2: 0.299505
[2469]	training's l2: 0.299504
[2470]	training's l2: 0.299501
[2471]	training's l2: 0.2995
[2472]	train

[2705]	training's l2: 0.299121
[2706]	training's l2: 0.299119
[2707]	training's l2: 0.299118
[2708]	training's l2: 0.299116
[2709]	training's l2: 0.299115
[2710]	training's l2: 0.299114
[2711]	training's l2: 0.299113
[2712]	training's l2: 0.299111
[2713]	training's l2: 0.299109
[2714]	training's l2: 0.299107
[2715]	training's l2: 0.299107
[2716]	training's l2: 0.299106
[2717]	training's l2: 0.299105
[2718]	training's l2: 0.299105
[2719]	training's l2: 0.299103
[2720]	training's l2: 0.299102
[2721]	training's l2: 0.2991
[2722]	training's l2: 0.299099
[2723]	training's l2: 0.299098
[2724]	training's l2: 0.299097
[2725]	training's l2: 0.299096
[2726]	training's l2: 0.299096
[2727]	training's l2: 0.299094
[2728]	training's l2: 0.299094
[2729]	training's l2: 0.299093
[2730]	training's l2: 0.299092
[2731]	training's l2: 0.29909
[2732]	training's l2: 0.29909
[2733]	training's l2: 0.299088
[2734]	training's l2: 0.299087
[2735]	training's l2: 0.299086
[2736]	training's l2: 0.299084
[2737]	train

[2971]	training's l2: 0.298851
[2972]	training's l2: 0.29885
[2973]	training's l2: 0.298849
[2974]	training's l2: 0.298848
[2975]	training's l2: 0.298848
[2976]	training's l2: 0.298847
[2977]	training's l2: 0.298847
[2978]	training's l2: 0.298845
[2979]	training's l2: 0.298845
[2980]	training's l2: 0.298844
[2981]	training's l2: 0.298842
[2982]	training's l2: 0.298841
[2983]	training's l2: 0.298841
[2984]	training's l2: 0.298839
[2985]	training's l2: 0.298838
[2986]	training's l2: 0.298837
[2987]	training's l2: 0.298836
[2988]	training's l2: 0.298835
[2989]	training's l2: 0.298834
[2990]	training's l2: 0.298833
[2991]	training's l2: 0.298832
[2992]	training's l2: 0.298831
[2993]	training's l2: 0.29883
[2994]	training's l2: 0.298829
[2995]	training's l2: 0.298828
[2996]	training's l2: 0.298828
[2997]	training's l2: 0.298828
[2998]	training's l2: 0.298826
[2999]	training's l2: 0.298826
[3000]	training's l2: 0.298825
[3001]	training's l2: 0.298824
[3002]	training's l2: 0.298823
[3003]	tra